In [1]:
# Notebook baseado em: CNES/CNES_LT.ipynb

# Passo 1: Instalar o PySUS (executar no Google Colab)
!pip install git+https://github.com/AlertaDengue/PySUS.git --upgrade

  Cloning https://github.com/AlertaDengue/PySUS.git to /tmp/pip-req-build-8_qa5w3i
  Running command git clone --filter=blob:none --quiet https://github.com/AlertaDengue/PySUS.git /tmp/pip-req-build-8_qa5w3i
  Resolved https://github.com/AlertaDengue/PySUS.git to commit 54f7800db4da5171d55e3446d638d64beeb8b149
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 k

# Antes de prosseguir, certifique-se de reiniciar a sessão clicando em
# Runtime -> Restart session
ou  
# Tempo de execução -> Reiniciar sessão

In [1]:
# Passo 2: IMPORTAR bibliotecas
from pysus.online_data.CNES import download
import pandas as pd

In [2]:
# Passo 3: Baixar os dados de leitos SUS do Paraná (jan/2023)
dados = download('LT', 'PR', 2023, 1)
df = dados.to_dataframe()

# Converter QT_SUS para número
df['QT_SUS'] = pd.to_numeric(df['QT_SUS'], errors='coerce')

# Agrupar por código do município
leitos_por_municipio = df.groupby('CODUFMUN')['QT_SUS'].sum().reset_index()
leitos_por_municipio['CODUFMUN'] = leitos_por_municipio['CODUFMUN'].astype(str).str.zfill(6)

LTPR2301.parquet: 100%|██████████| 1.80k/1.80k [00:00<00:00, 6.25kB/s]


In [3]:
# Passo 4: Carregar tabela de municípios atualizada do GitHub
url_municipios = "https://github.com/cartaproale/PySUS/raw/refs/heads/main/notebooks/Tabela_municipios.csv"
municipios = pd.read_csv(url_municipios, sep=';', encoding='latin1')
municipios = municipios.iloc[:, [1, 3]]  # Código IBGE + Nome IBGE
municipios.columns = ['CODUFMUN', 'municipio_nome']
municipios['CODUFMUN'] = municipios['CODUFMUN'].astype(str).str[:6]

# Corrigir a acentuação dos nomes de municípios
municipios['municipio_nome'] = municipios['municipio_nome'].apply(lambda x: x.encode('latin1').decode('utf-8') if isinstance(x, str) else x)

In [4]:
# Merge com nomes dos municípios
tabela_final = leitos_por_municipio.merge(municipios, on='CODUFMUN', how='left')
tabela_final = tabela_final[['CODUFMUN', 'municipio_nome', 'QT_SUS']].sort_values(by='QT_SUS', ascending=False)

# Exibir top 10
print("Top 10 municípios com mais leitos SUS no Paraná (jan/2023):")
print(tabela_final.head(10))

Top 10 municípios com mais leitos SUS no Paraná (jan/2023):
    CODUFMUN         municipio_nome  QT_SUS
57    410690               Curitiba    2959
113   411370               Londrina    1406
30    410420            Campo Largo    1187
124   411520                Maringá    1067
37    410480               Cascavel     705
231   412810               Umuarama     625
163   411990           Ponta Grossa     535
159   411950              Piraquara     501
29    410400  Campina Grande do Sul     411
12    410150              Arapongas     390


In [5]:
# Passo 5: Mostrar leitos SUS em Cascavel (PR)
cascavel = tabela_final[tabela_final['CODUFMUN'] == '410480']
print("\nLeitos SUS em Cascavel (PR) em jan/2023:")
print(cascavel)


Leitos SUS em Cascavel (PR) em jan/2023:
   CODUFMUN municipio_nome  QT_SUS
37   410480       Cascavel     705


In [6]:
# Passo 6: Calcular total de leitos SUS no estado
total_estado = tabela_final['QT_SUS'].sum()
print("\nTotal de leitos SUS no Paraná em jan/2023:", int(total_estado))


Total de leitos SUS no Paraná em jan/2023: 20721
